In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm # 建模库
from sklearn.preprocessing import StandardScaler
# 自定义设置 # 添加根目录路径到搜索列表
import sys
sys.path.append(r'C:\Users\86199\Documents\GitHub\DataAnalysis')
from my_setting import my_pd_setting,my_np_setting,my_sns_setting,data_path
from my_stat import get_col_dtypes, missing, drop_columns, des_classifi
my_pd_setting()
my_sns_setting()
my_np_setting()
filepath= r'C:\Users\86199\Documents\Datas\DC\C1'
filename = data_path(filepath=filepath)

pandas的版本为: 1.3.4
seaborn的版本为: 0.11.2
numpy的版本为1.20.3
已切换至数据路径: 
c:\Users\86199\Documents\GitHub\DataAnalysis\DC\C1
数据路径存在文件: 
['ols_out1.csv', 'submit_example.csv', 'test_noLabel.csv', 'train.csv']


In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test_noLabel.csv')
train.head(2)

,ID,位置,出租方式,区,卧室数量,卫的数量,厅的数量,地铁站点,地铁线路,小区名,小区房屋出租数量,居住状态,总楼层,房屋朝向,房屋面积,时间,楼层,装修情况,距离,Label
0,0,118.0,NaN,11.0,1,1,1,4.0,2.0,3072,128.91,NaN,23.64,东南,86.28,1,2,NaN,76.42,5.60
1,1,100.0,NaN,10.0,1,0,0,5.8,4.0,3152,132.81,NaN,38.18,东,170.46,1,1,NaN,70.92,16.98


##### 变量信息，数据类型

In [3]:
dtype = get_col_dtypes(train)
dtype.T

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196539 entries, 0 to 196538
Columns: 20 entries, ID to Label
dtypes: float64(12), int64(7), object(1)
memory usage: 30.0+ MB
None


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
var_names,ID,位置,出租方式,区,卧室数量,卫的数量,厅的数量,地铁站点,地铁线路,小区名,小区房屋出租数量,居住状态,总楼层,房屋朝向,房屋面积,时间,楼层,装修情况,距离,Label
dtypes,int64,float64,float64,float64,int64,int64,int64,float64,float64,int64,float64,float64,float64,object,float64,int64,int64,float64,float64,float64


##### 变量的缺失值情况

In [4]:
miss = missing(train, choice=False)
miss.T

,17,11,2,18,7,8,10,3,1,13,16,15,14,0,12,9,6,5,4,19
var_name,装修情况,居住状态,出租方式,距离,地铁站点,地铁线路,小区房屋出租数量,区,位置,房屋朝向,楼层,时间,房屋面积,ID,总楼层,小区名,厅的数量,卫的数量,卧室数量,Label
missing,178047,176401,172309,104761,104761,104761,1001,31,31,0,0,0,0,0,0,0,0,0,0,0
per,90.59,89.75,87.67,53.3,53.3,53.3,0.51,0.02,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
miss = missing(train, choice=True)
miss.T

缺失值var_name: ['装修情况', '居住状态', '出租方式', '距离', '地铁站点', '地铁线路', '小区房屋出租数量', '区', '位置']


,0,1,2,3,4,5,6,7,8
var_name,装修情况,居住状态,出租方式,距离,地铁站点,地铁线路,小区房屋出租数量,区,位置
missing,178047,176401,172309,104761,104761,104761,1001,31,31
per,90.59,89.75,87.67,53.3,53.3,53.3,0.51,0.02,0.02


##### 方案1，对缺失变量进行删除处理

In [6]:
columns = miss['var_name'].tolist()
columns.append('小区名')
print('''需要删除的列:  ''')
print(columns)
train = drop_columns(train, columns=columns)
test = drop_columns(test, columns=columns)
train.head(3)

需要删除的列:  
['装修情况', '居住状态', '出租方式', '距离', '地铁站点', '地铁线路', '小区房屋出租数量', '区', '位置', '小区名']


,ID,卧室数量,卫的数量,厅的数量,总楼层,房屋朝向,房屋面积,时间,楼层,Label
0,0,1,1,1,23.64,东南,86.28,1,2,5.60
1,1,1,0,0,38.18,东,170.46,1,1,16.98
2,2,2,2,1,29.09,东南,105.93,1,0,9.00


##### 将ID设置为索引,删除不需要的列；再次查看剩余变量的数据类型

In [7]:
train.set_index('ID', inplace=True)
test.set_index('ID', inplace=True)
train.head(3)

,卧室数量,卫的数量,厅的数量,总楼层,房屋朝向,房屋面积,时间,楼层,Label
ID,,,,,,,,,
0,1,1,1,23.64,东南,86.28,1,2,5.60
1,1,0,0,38.18,东,170.46,1,1,16.98
2,2,2,1,29.09,东南,105.93,1,0,9.00


In [8]:
dtype = get_col_dtypes(train)
dtype.T

<class 'pandas.core.frame.DataFrame'>
Int64Index: 196539 entries, 0 to 196538
Columns: 9 entries, 卧室数量 to Label
dtypes: float64(3), int64(5), object(1)
memory usage: 15.0+ MB
None


,0,1,2,3,4,5,6,7,8
var_names,卧室数量,卫的数量,厅的数量,总楼层,房屋朝向,房屋面积,时间,楼层,Label
dtypes,int64,int64,int64,float64,object,float64,int64,int64,float64


In [9]:
des = des_classifi(train,y='Label', columns=['时间', '楼层'])

分类变量个数: 2
----------------------------------------
变量 时间 的描述统计结果:  
   时间  cates_mean
0  1   8.10      
1  2   7.94      
2  3   7.86      
----------------------------------------
变量 楼层 的描述统计结果:  
   楼层  cates_mean
0  0   7.79      
1  1   8.23      
2  2   7.91      
----------------------------------------
